In [2]:
import pandas as pd
import sys
import warnings

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

* File reading with video mapping

In [3]:
df_map_videos_file = pd.read_csv("/IPTV_Datasets_28Jan/" + "match_files_sorted_mapped" + ".csv")
df_map_videos_file.head(5)

,date,time,filename,foldername,channel,location,codec,flag,group_time
0,2022-02-02,11-31-48,tmplist5_pt_sportv1hevc_without_vpn_2u9q3bq1f9...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,0,0
1,2022-02-02,11-41-38,tmplist5_pt_sportv1hevc_without_vpn_3x4712uxky...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,1,0
2,2022-02-02,14-27-14,tmplist5_pt_sportv1hevc_without_vpn_5fu61xjunp...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,2,0
3,2022-02-02,13-31-39,tmplist5_pt_sportv1hevc_without_vpn_5qjsk2qnp8...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,2,0
4,2022-02-02,14-21-32,tmplist5_pt_sportv1hevc_without_vpn_5vmdwnyjsn...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,2,0


In [4]:
df_map_videos_file.reset_index(drop=True, inplace=True)

# Remove rows without_vpn
linhas_remover = []
for idx, row in df_map_videos_file.iterrows():
    file_name = row["filename"].split(".")[0]  #remove .csv from filename
    file_name_new = file_name.replace(':', '_')
    if file_name_new.split("_")[3] == "without":
        linhas_remover.append(idx)
df_map_videos_file_clean = df_map_videos_file.drop(linhas_remover)

print(len(df_map_videos_file_clean))
df_map_videos_file_clean.head(5)

136


,date,time,filename,foldername,channel,location,codec,flag,group_time
23,2022-02-02,11-31-48,tmplist5_pt_sportv1hevc_with_vpn_2u9q3bq1f9vgq...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,0,1
24,2022-02-02,11-41-38,tmplist5_pt_sportv1hevc_with_vpn_3x4712uxkyfun...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,1,1
25,2022-02-02,14-27-14,tmplist5_pt_sportv1hevc_with_vpn_5fu61xjunpkpj...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,2,1
26,2022-02-02,13-31-39,tmplist5_pt_sportv1hevc_with_vpn_5qjsk2qnp81xy...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,2,2
27,2022-02-02,14-21-32,tmplist5_pt_sportv1hevc_with_vpn_5vmdwnyjsn1ns...,tmplist5_sporttv1_pt_hevc,sporttv1,pt,hevc,2,2


* Create Histograms

In [ ]:
if 'histograms' in sys.modules:
    del sys.modules["histograms"]
from create_histogramas_aux import histograms

# call function histograms from create_histograms_aux.py
histograms_list = histograms(df_map_videos_file_clean)

In [ ]:
# concat the histograms data and save to .csv file
all_histograms = pd.concat(histograms_list, ignore_index=True)
all_histograms.to_csv("IPTV_Datasets_28Jan/" + "all_histograms" + ".csv", index=False, header=True)

* If you already have the .csv whith all histograms, start from here

In [ ]:
all_histograms = pd.read_csv("/IPTV_Datasets_28Jan/" + "all_histograms" + ".csv")
all_histograms.head()

* Create scenarios and compute DTW

In [ ]:
# Import DTW Module
if 'dtw_aida' in sys.modules:
    del sys.modules["dtw_aida"]
from dtw_aida import compute_histogram_dtw, create_histogram_list
from datetime import datetime

In [ ]:
# All channels, codec hevec, location pt
df_rtp_hevc_pt = all_histograms.loc[(all_histograms['channel'] == 'rtp1') & (all_histograms['codec'] == 'hevc') & (all_histograms['location']=='pt')]
df_sportv_hevec_pt = all_histograms.loc[(all_histograms['channel'] == 'sporttv1') & (all_histograms['codec'] == 'hevc') & (all_histograms['location']=='pt')]

gk1 = df_rtp_hevc_pt.groupby(['10'])
gk2 = df_sportv_hevec_pt.groupby(['10'])

dataframes = [group for _, group in gk1]
dataframes2 = [group for _, group in gk2]
histograms_list = dataframes + dataframes2

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

df_ch_codec_loc = compute_histogram_dtw(histograms_list)
df_dtw_k_cen1 = pd.DataFrame(df_ch_codec_loc, columns=['vpn1', 'vpn2', 'dtw', 'euclidean','path', 'cost_matrix', 'x', 'y', 'diagonal_dtw', 'diagonal_euclidean'])
df_dtw_k_cen1.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario1" + ".csv", index=False, header=True)
df_dtw_k_cen1.head(5)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [41]:
# All channels, codec h264, location pt
df_rtp_h264_pt = all_histograms.loc[(all_histograms['channel'] == 'rtp1') & (all_histograms['codec'] == 'h264') & (all_histograms['location']=='pt')]
df_sportv_h264_pt = all_histograms.loc[(all_histograms['channel'] == 'sporttv1') & (all_histograms['codec'] == 'h264') & (all_histograms['location']=='pt')]
gk1 = df_rtp_h264_pt.groupby(['10'])
gk2 = df_sportv_h264_pt.groupby(['10'])

dataframes = [group for _, group in gk1]
dataframes2 = [group for _, group in gk2]
histograms_list = dataframes + dataframes2

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

df_ch_codec_loc = compute_histogram_dtw(histograms_list)
df_dtw_k_cen2 = pd.DataFrame(df_ch_codec_loc, columns=['vpn1', 'vpn2', 'dtw', 'euclidean','path', 'cost_matrix', 'x', 'y', 'diagonal_dtw', 'diagonal_euclidean'])
df_dtw_k_cen2.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario2" + ".csv", index=False, header=True)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 12:44:40
Current Time = 13:37:59


In [102]:
## All channels, all codecs, location pt
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_rtp_pt = all_histograms.loc[(all_histograms['channel'] == 'rtp1') & (all_histograms['location']=='pt')]
df_sportv_pt = all_histograms.loc[(all_histograms['channel'] == 'sporttv1') & (all_histograms['location']=='pt')]

if(df_rtp_pt.shape[0] > 0 and df_sportv_pt.shape[0] > 0):
    gk1 = df_rtp_pt.groupby(['10'])
    gk2 = df_sportv_pt.groupby(['10'])

    dataframes = [group for _, group in gk1]
    dataframes2 = [group for _, group in gk2]
    histograms_list = dataframes + dataframes2

    df_ch_codec_loc = compute_histogram_dtw(histograms_list)
    df_dtw_k = pd.DataFrame(df_ch_codec_loc, columns=['vpn1', 'vpn2', 'dtw', 'euclidean','path', 'cost_matrix', 'x', 'y', 'diagonal_dtw', 'diagonal_euclidean'])
    df_dtw_k.to_csv("/IPTV_Datasets_28Jan/" + "scenario6" + ".csv", index=False, header=True)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

Current Time = 13:12:03
Current Time = 16:05:39


In [12]:
## All channels, all codecs, location uk
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_rtp = all_histograms.loc[(all_histograms['channel'] == 'rtp1')]
df_sportv = all_histograms.loc[(all_histograms['channel'] == 'sporttv1')]

hist_list = create_histogram_list(df_rtp, df_sportv)
df_ch_codec_loc = compute_histogram_dtw(hist_list)
df_dtw_k = pd.DataFrame(df_ch_codec_loc,
            columns=['vpn1', 'vpn2', 'dtw', 'euclidean', 'path', 'cost_matrix', 'x', 'y',
            'diagonal_dtw', 'diagonal_euclidean'])
df_dtw_k.to_csv("/IPTV_Datasets_28Jan/" + "dtw_cenario7" + ".csv", index=False, header=True)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

Current Time = 23:17:42
Current Time = 05:35:33


In [13]:
## DTW by codec
# Same channel, different codecs
# channel rtp, codecs h264 e hevec

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_rpt_h264 = all_histograms.loc[(all_histograms['channel'] == 'rtp1') & (all_histograms['codec'] == 'h264')]
df_rtp_hevc = all_histograms.loc[(all_histograms['channel'] == 'rtp1') & (all_histograms['codec'] == 'hevc')]

hist_list = create_histogram_list(df_rpt_h264, df_rtp_hevc)
df_ch_codec_loc = compute_histogram_dtw(hist_list)
df_dtw_k = pd.DataFrame(df_ch_codec_loc,
            columns=['vpn1', 'vpn2', 'dtw', 'euclidean', 'path', 'cost_matrix', 'x', 'y',
            'diagonal_dtw', 'diagonal_euclidean'])
df_dtw_k.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario8" + ".csv", index=False, header=True)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

Current Time = 05:35:33
Current Time = 06:17:39


In [15]:
# channel sportv, codecs h264 e hevec

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_sptv_hevec = all_histograms.loc[(all_histograms['channel'] == 'sporttv1') & (all_histograms['codec'] == 'h264')]
df_sptv_h264 = all_histograms.loc[(all_histograms['channel'] == 'sporttv1') & (all_histograms['codec'] == 'hevc')]

hist_list = create_histogram_list(df_sptv_hevec, df_sptv_h264)
df_ch_codec_loc = compute_histogram_dtw(hist_list)
df_dtw_k = pd.DataFrame(df_ch_codec_loc,
            columns=['vpn1', 'vpn2', 'dtw', 'euclidean', 'path', 'cost_matrix', 'x', 'y',
            'diagonal_dtw', 'diagonal_euclidean'])
df_dtw_k.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario9" + ".csv", index=False, header=True)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))


Current Time = 06:39:06
Current Time = 09:17:57


In [16]:
# Channel sportv1, all locations
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_sptv_uk = all_histograms.loc[(all_histograms['channel'] == 'sporttv1') & (all_histograms['location'] == 'uk')]
df_sptv_pt = all_histograms.loc[(all_histograms['channel'] == 'sporttv1') & (all_histograms['location'] == 'pt')]

hist_list = create_histogram_list(df_sptv_uk, df_sptv_pt)
df_ch_codec_loc = compute_histogram_dtw(hist_list)
df_dtw_k = pd.DataFrame(df_ch_codec_loc,
            columns=['vpn1', 'vpn2', 'dtw', 'euclidean', 'path', 'cost_matrix', 'x', 'y',
            'diagonal_dtw', 'diagonal_euclidean'])
df_dtw_k.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario10" + ".csv", index=False, header=True)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

Current Time = 13:37:54
Current Time = 16:42:08


In [17]:
# Channel rtp1, all locations
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_rtp_uk = all_histograms.loc[(all_histograms['channel'] == 'rtp1') & (all_histograms['location'] == 'uk')]
df_rtp_pt = all_histograms.loc[(all_histograms['channel'] == 'rtp1') & (all_histograms['location'] == 'pt')]

hist_list = create_histogram_list(df_rtp_uk, df_rtp_pt)
df_ch_codec_loc = compute_histogram_dtw(hist_list)
df_dtw_k = pd.DataFrame(df_ch_codec_loc,
            columns=['vpn1', 'vpn2', 'dtw', 'euclidian', 'path', 'cost_matrix', 'x', 'y',
            'diagonal_dtw', 'diagonal_euclidian'])
df_dtw_k.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario11" + ".csv", index=False, header=True)
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))



Current Time = 09:38:00


TypeError: 'NoneType' object is not iterable

In [ ]:
# DTW by location
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_uk = all_histograms.loc[(all_histograms['location'] == 'uk')]
df_pt = all_histograms.loc[(all_histograms['location'] == 'pt')]

hist_list = create_histogram_list(df_uk, df_pt)
df_ch_codec_loc = compute_histogram_dtw(hist_list)
df_dtw_k = pd.DataFrame(df_ch_codec_loc,
            columns=['vpn1', 'vpn2', 'dtw', 'euclidean', 'path', 'cost_matrix', 'x', 'y',
            'diagonal_dtw', 'diagonal_euclidean'])
df_dtw_k.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario12" + ".csv", index=False, header=True)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

In [73]:
## DTW por codecs
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

df_hevec_cen13 = all_histograms.loc[(all_histograms['codec'] == 'hevc')]
df_h264_cen13 = all_histograms.loc[(all_histograms['codec'] == 'h264')]

df_hevec_cen13_gk1 = df_hevec_cen13.groupby(['10'])
df_h264_cen13_gk2 = df_h264_cen13.groupby(['10'])

dataframes = [group for _, group in df_hevec_cen13_gk1]
dataframes2 = [group for _, group in df_h264_cen13_gk2]

histograms_list_cen13 = dataframes + dataframes2

df_hist_cen13 = compute_histogram_dtw(histograms_list_cen13)
df_dtw_k_cen13 = pd.DataFrame(df_hist_cen13, columns=['vpn1', 'vpn2', 'dtw', 'euclidian','path', 'cost_matrix', 'x', 'y', 'diagonal_dtw', 'diagonal_euclidian'])
df_dtw_k_cen13.to_csv("/IPTV_Datasets_28Jan/" + "dtw_scenario13_codecs" + ".csv", index=False, header=True)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))



Current Time = 09:57:14


,0,1,2,3,4,5,6,7,8,9,10,11,foldername,channel,location,codec,group_time,time
6661,0.100000,0.0,0.000000,0.300000,0.0,0.500000,0.0,0.0,0.0,0.100000,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6662,0.431890,0.0,0.000000,0.000628,0.0,0.000628,0.0,0.0,0.0,0.566855,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6663,0.371265,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.628735,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6664,0.276224,0.0,0.000000,0.000000,0.0,0.003497,0.0,0.0,0.0,0.720280,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6665,0.351724,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.648276,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6666,0.304207,0.0,0.000000,0.000000,0.0,0.003236,0.0,0.0,0.0,0.692557,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6667,0.353211,0.0,0.002294,0.000000,0.0,0.000000,0.0,0.0,0.0,0.644495,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6668,0.322034,0.0,0.000000,0.000000,0.0,0.003390,0.0,0.0,0.0,0.674576,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6669,0.301226,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.698774,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
6670,0.259868,0.0,0.000000,0.000000,0.0,0.003289,0.0,0.0,0.0,0.736842,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_pt:sportv1fhd_with_vpn_28psrzdcbgfkwu...,tmplist2_sporttv1_pt_h264,sporttv1,pt,h264,2,13-50-58
